In [ ]:
# get the image to process
# todo: select 10 random images from other pile, possibly batch
filename = "IMG_2712"

In [ ]:
import base64
import google.generativeai as genai

# load the system instructions and prompt from markdown files
with open('prompts/system_instructions_no_ocr.md', 'r') as f:
   system_instructions = f.read()
with open('prompts/few_shot_prompt_no_ocr.md', 'r') as f:
   prompt = f.read()


# Read the image as bytes and encode to base64
def make_image_part(image_path):
    with open(image_path, "rb") as img_file:
        img_bytes = img_file.read()
    return {
        "mime_type": "image/jpeg",
        "data": img_bytes,
    }


example_part = make_image_part("../img/IMG_2708.jpg")
classify_part = make_image_part(f"../img/{filename}.jpg")


In [ ]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()

genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

model_name = "gemini-2.5-pro-preview-05-06"

model = genai.GenerativeModel(
    model_name,
    system_instruction=system_instructions
)
response = model.generate_content(contents=[example_part, classify_part, prompt])



In [ ]:
import re
import json

response_text = response.candidates[0].content.parts[0].text
# Convert response to JSON-serializable format
response_serializable = re.sub(r'^```json\s*|```$', '', response_text.strip())

# Save to file
output_path = os.path.join(os.path.abspath('..'), "tmp", "herbarium_processed_output" + filename + ".json")

with open(output_path, 'w') as f:
    json.dump(response_serializable, f, indent=2)

print(f"Saved output to {output_path}")
print(response_serializable)